In [24]:
import pandas as pd

In [41]:
train_df=pd.read_csv('Train_Data.csv')
test_df=pd.read_csv('Test_Data.csv')
train_df.head(10)

,SEQN,RIAGENDR,PAQ605,BMXBMI,LBXGLU,DIQ010,LBXGLT,LBXIN,age_group
0,73564.0,2.0,2.0,35.7,110.0,2.0,150.0,14.91,Adult
1,73568.0,2.0,2.0,20.3,89.0,2.0,80.0,3.85,Adult
2,73576.0,1.0,2.0,23.2,89.0,2.0,68.0,6.14,Adult
3,73577.0,1.0,2.0,28.9,104.0,NaN,84.0,16.15,Adult
4,73580.0,2.0,1.0,35.9,103.0,2.0,81.0,10.92,Adult
5,73581.0,1.0,2.0,23.6,110.0,2.0,100.0,6.08,Adult
6,73587.0,1.0,2.0,38.7,94.0,2.0,202.0,21.11,Adult
7,73596.0,2.0,2.0,38.3,107.0,2.0,164.0,20.93,Adult
8,73607.0,1.0,2.0,38.9,89.0,2.0,113.0,17.47,Senior
9,73610.0,1.0,1.0,28.9,90.0,2.0,95.0,3.24,Adult


In [26]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.impute import KNNImputer

In [27]:
train_df.drop('SEQN',axis=1,inplace=True)
train_df['Is_Senior']=[0 if x =='Adult' else 1 for x in train_df['age_group']]
train_df.drop('age_group',axis=1,inplace=True)

target_col=['Is_Senior']
feature_cols=[col for col in train_df.columns if col not in target_col]


In [28]:
it_imputer= KNNImputer()
train_df[feature_cols]=it_imputer.fit_transform(train_df[feature_cols])

In [29]:
import numpy as np

def detect_outliers(data):
    
    q1,q3=np.percentile(data,[25,75])
    
    iqr=q3-q1
    lower_bound=q1-(1.5*iqr)
    upper_bound=q3+(1.5*iqr)
    
    data = np.array(data, dtype=float)

    data[(data < lower_bound) | (data > upper_bound)] = np.nan

    data=pd.Series(data)
    
    return data
    

In [30]:
train_df['BMXBMI']= detect_outliers(train_df['BMXBMI'])

In [31]:
train_df['LBXGLU'] = detect_outliers(train_df['LBXGLU'])
train_df['LBXGLT'] = detect_outliers(train_df['LBXGLT'])
train_df['LBXIN'] = detect_outliers(train_df['LBXIN'])

In [32]:
train_df[feature_cols]=it_imputer.fit_transform(train_df[feature_cols])

In [33]:
train_df.isnull().sum()

RIAGENDR     0
PAQ605       0
BMXBMI       0
LBXGLU       0
DIQ010       0
LBXGLT       0
LBXIN        0
Is_Senior    0
dtype: int64

In [34]:
train_df.loc[train_df['PAQ605'] > 2, 'PAQ605'] = 2


In [35]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train_df[feature_cols],train_df[target_col],test_size=0.2)

In [36]:
from sklearn.ensemble import RandomForestClassifier
model2=RandomForestClassifier(
    n_estimators=250,
    min_samples_split=5,     
    min_samples_leaf=3,      
    max_depth=10,            
    class_weight='balanced',
    random_state=42
    )

model2.fit(X_train,y_train)

c:\Users\ujjwal krishna\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(class_weight='balanced', max_depth=10,
                       min_samples_leaf=3, min_samples_split=5,
                       n_estimators=250, random_state=42)

In [37]:
model2.score(X_train,y_train)

0.898854961832061

In [38]:
test_df.drop('SEQN',axis=1,inplace=True)

test_df[feature_cols]=it_imputer.fit_transform(test_df[feature_cols])

test_df['BMXBMI']= detect_outliers(test_df['BMXBMI'])
test_df['LBXGLU'] = detect_outliers(test_df['LBXGLU'])
test_df['LBXGLT'] = detect_outliers(test_df['LBXGLT'])
test_df['LBXIN'] = detect_outliers(test_df['LBXIN'])

test_df[feature_cols]=it_imputer.fit_transform(test_df[feature_cols])

test_df.loc[test_df['PAQ605'] > 2, 'PAQ605'] = 2

In [39]:
y_pred = model2.predict(test_df)

In [40]:
result = pd.DataFrame({
    'age_group': y_pred
})

result.to_csv('result4.csv',index=False)